In [2]:
using Distributions, QuadGK, PyPlot

In [34]:
a = 3
b = -.5
nu = 1

a = 2
b = -1
nu = 1


function LOO(Gamma)
    pdf_odd(t)  = pdf(Normal(a, sqrt(2/nu)), t)
    pdf_even(t) = pdf(Normal(b, sqrt(2/nu)), t)

    I1, E1 = quadgk(t-> a/2 * min(1, sqrt(nu)/Gamma / sqrt(2) * max(0, t)) * pdf_odd(t), -5, 5)
    I2, E2 = quadgk(t-> b/2 * min(1, sqrt(nu)/Gamma / sqrt(2) * max(0, t)) * pdf_even(t), -5, 5)
    I1 + I2
end

# CDF(t) = cdf(Normal(), t)
# PDF(t) = pdf(Normal(), t)

# function LOO2(Gamma)
#     a/2 * ((a-1) * CDF(Gamma - a * sqrt(nu)/sqrt(2)) - sqrt(2)/sqrt(nu) * PDF(Gamma - a* sqrt(nu)/sqrt(2)) ) + 
#     b/2 * ((b-1) * CDF(Gamma - b * sqrt(nu)/sqrt(2)) - sqrt(2)/sqrt(nu) * PDF(Gamma - b* sqrt(nu)/sqrt(2)) )     
# end

function OR(Gamma)
    pdf_odd(t) = pdf(Normal(a, sqrt(1/nu)), t)
    pdf_even(t) = pdf(Normal(b, sqrt(1/nu)), t)

    I1, E1 = quadgk(t-> a/2 * min(1, sqrt(nu)/Gamma * max(0, t)) * pdf_odd(t), -5, 5)
    I2, E2 = quadgk(t-> b/2 * min(1, sqrt(nu)/Gamma * max(0, t)) * pdf_even(t), -5, 5)
    I1 + I2
end

Gamma_grid = collect(linspace(.00001, 1, 1000))
out = map(LOO, Gamma_grid);
out2 = map(OR, Gamma_grid);

maximum(out2) - out2[indmax(out)]


0.005610527441543445

In [35]:
Gamma_grid[indmax(out)]

#writecsv("temp.csv", [Gamma_grid out out2 ])

0.04004963963963964

In [ ]:
plot(Gamma_grid, out)